In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
import time

from sklearn.metrics import log_loss

path = Path('./')

submission = pd.read_csv(path / 'sample_submission.csv', index_col='id')
labels = pd.read_csv(path / 'train_labels.csv', index_col='id')

# the ids of the submission rows (useful later)
sub_ids = submission.index

# the ids of the labeled rows (useful later)
gt_ids = labels.index 

# list of files in the submission folder
subs = sorted(os.listdir(path / 'submission_files'))
print(len(subs))

5000


In [2]:
np_preds = np.zeros((40000,len(subs)))
df_preds = pd.DataFrame(np_preds)
scores = []
start = time.time()
for i,sub in enumerate(subs):
    pred = pd.read_csv(path / "submission_files" / sub, index_col = "id",encoding = "shift-jis")
    # np_preds[:,i] = pred["pred"]
    df_preds.iloc[:,i] = pred["pred"]
end = time.time()

print(end-start)
np_preds = df_preds.to_numpy()

66.74738097190857


## Data set

In [8]:
from sklearn.model_selection import train_test_split as tts

data_x = np_preds[:len(labels),:]
data_y = labels
test_x = np_preds[len(labels):,:]

df_x = df_preds.iloc[:len(labels),:]
df_y = labels[["label"]]
df_test_x = df_preds.iloc[len(labels):,:]

train_x,val_x,train_y,val_y=tts(data_x,data_y,random_state=10,test_size=0.3)

ensemble_set = data_y.copy()


In [25]:
ensemble_set.head()

,label
id,
0,0
1,1
2,1
3,1
4,0


## Model1

In [33]:
import lightgbm as lgb

In [34]:
params = {
    "objective" : "binary",
    "metric" : "binary_logloss",
    "random_state":10,
    "learning_rate":0.05,
    "max_depth":3
}
num_round = 100

lgb_train = lgb.Dataset(train_x,train_y)
lgb_val = lgb.Dataset(val_x,val_y)

# train
model = lgb.train(params,lgb_train,num_boost_round=num_round, valid_names=['train', 'valid']
                  , valid_sets=[lgb_train, lgb_val], early_stopping_rounds=20)

/Users/komachiyuuki/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 6993, number of negative: 7007
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 1.577276 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1261109
[LightGBM] [Info] Number of data points in the train set: 14000, number of used features: 4998
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499500 -> initscore=-0.002000
[LightGBM] [Info] Start training from score -0.002000
[1]	train's binary_logloss: 0.67911	valid's binary_logloss: 0.679654
Training until validation scores don't improve for 20 rounds
[2]	train's binary_logloss: 0.666381	valid's binary_logloss: 0.667449
[3]	train's binary_logloss: 0.654848	valid's binary_logloss: 0.656376
[4]	train's binary_logloss: 0.644327	valid's binary_logloss: 0.646056
[5]	train's binary_logloss: 0.634706	valid's binary_logloss: 0.636628
[6]	train's binary_logloss: 0.625882	valid's binary_logloss: 0.628042
[7]	train's binary_l

In [ ]:
blend = model.predict(test_x)
submission["pred"] = blend
submission.reset_index().to_csv("model1.csv",index = False)

## Model2

In [ ]:
params = {
    "objective" : "binary",
    "metric" : "binary_logloss",
    "random_state":10,
    "learning_rate":0.05,
    "max_depth":6,
    "num_leaves":64
}
num_round = 100

lgb_train = lgb.Dataset(train_x,train_y)
lgb_val = lgb.Dataset(val_x,val_y)

# train
model = lgb.train(params,lgb_train,num_boost_round=num_round, valid_names=['train', 'valid']
                  , valid_sets=[lgb_train, lgb_val], early_stopping_rounds=20)

In [ ]:
blend = model.predict(test_x)
submission["pred"] = blend
submission.reset_index().to_csv("model2.csv",index = False)

## Model3

In [15]:
params = {
    "objective" : "binary",
    "metric" : "binary_logloss",
    "random_state":10,
    "learning_rate":0.05,
    "max_depth":9,
    "num_leaves":512
}
num_round = 150

lgb_train = lgb.Dataset(train_x,train_y)
lgb_val = lgb.Dataset(val_x,val_y)

# train
model = lgb.train(params,lgb_train,num_boost_round=num_round, valid_names=['train', 'valid']
                  , valid_sets=[lgb_train, lgb_val], early_stopping_rounds=20)

/Users/komachiyuuki/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 6993, number of negative: 7007
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 1.189347 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1261109
[LightGBM] [Info] Number of data points in the train set: 14000, number of used features: 4998
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499500 -> initscore=-0.002000
[LightGBM] [Info] Start training from score -0.002000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	train's binary_logloss: 0.673195	valid's binary_logloss: 0.67971
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	train's binary_logloss: 0.655214	valid's binary_logloss: 0.66723
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	train's binary_logloss: 0.639591	valid's binary_logloss: 0.655892
[Light

In [16]:
blend = model.predict(test_x)
submission["pred"] = blend
submission.reset_index().to_csv("model3.csv",index = False)

## Model4

## Stacking

In [71]:
from sklearn.model_selection import KFold

def create_stacking_feature(model,data_x,data_y,test_x,mode=0,params=0):#mode{0:lightgbmなどのDatasetを作るもの,1:その他}
    splits = 5
    pred_train = pd.DataFrame(np.zeros((len(data_x),1)))
    pred_test = []
    pred_train.columns = ['pred'] 
    cv = KFold(n_splits=splits,shuffle=True,random_state=10)
    if(mode==0):
        for i,(train_index,target_index) in enumerate(cv.split(data_x)):
            train_x,val_x,train_y,val_y = tts(data_x.iloc[train_index,:],data_y.iloc[train_index],test_size=0.25,random_state=10)
            target_x,target_y = data_x.iloc[target_index,:],data_y.iloc[target_index]
            
            lgb_train = lgb.Dataset(train_x,train_y)
            lgb_val = lgb.Dataset(val_x,val_y)
            
            model = lgb.train(params,
                              lgb_train,
                              num_boost_round=150,
                              valid_names=['train', 'valid'],
                              valid_sets=[lgb_train, lgb_val], 
                              callbacks=[lgb.early_stopping(stopping_rounds=20)])
            pred_train['pred'].iloc[target_index] = model.predict(target_x)
            pred_test.append(model.predict(test_x))
            
    else:
        for train_index,test_index in cv(data_x):
            print('未実装')
            break
    pred_test = np.mean(pred_test,axis=0)
    pred_test = pd.DataFrame(pred_test)
    pred_test.columns = ['pred']
    
    return pred_train,pred_test
        
        
        
        

In [105]:
params = {
    "objective" : "binary",
    "metric" : "binary_logloss",
    "random_state":10,
    "learning_rate":0.05,
    "max_depth":3,
    'num_leaves':7
}
tr1,pr1=create_stacking_feature(model=0,data_x=df_x,data_y=df_y,test_x=df_test_x,mode=0,params=params)

params2 = {
    "objective" : "binary",
    "metric" : "binary_logloss",
    "random_state":10,
    "learning_rate":0.05,
    "max_depth":6,
    'num_leaves':63
}
tr2,pr2=create_stacking_feature(model=0,data_x=df_x,data_y=df_y,test_x=df_test_x,mode=0,params=params2)

params3 = {
    "objective" : "binary",
    "metric" : "binary_logloss",
    "random_state":10,
    "learning_rate":0.05,
    "max_depth":9,
    'num_leaves':512

}
tr3,pr3=create_stacking_feature(model=0,data_x=df_x,data_y=df_y,test_x=df_test_x,mode=0,params=params3)

params3 = {
    "objective" : "binary",
    "metric" : "binary_logloss",
    "random_state":10,
    "learning_rate":0.05,
    "max_depth":9,
    'num_leaves':512

}
tr3,pr3=create_stacking_feature(model=0,data_x=df_x,data_y=df_y,test_x=df_test_x,mode=0,params=params3)

params4 = {
    "objective" : "binary",
    "metric" : "binary_logloss",
    "random_state":10,
    "learning_rate":0.05,
    "max_depth":5,
    'num_leaves':64,
    'lambda_l1':2
}
tr4,pr4=create_stacking_feature(model=0,data_x=df_x,data_y=df_y,test_x=df_test_x,mode=0,params=params4)

params5 = {
    "objective" : "binary",
    "metric" : "binary_logloss",
    "random_state":10,
    "learning_rate":0.05,
    "max_depth":5,
    'num_leaves':64,
    'lambda_l1':4
}
tr5,pr5=create_stacking_feature(model=0,data_x=df_x,data_y=df_y,test_x=df_test_x,mode=0,params=params5)

[LightGBM] [Info] Number of positive: 5965, number of negative: 6035
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 1.669976 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1261021
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 4998
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.497083 -> initscore=-0.011667
[LightGBM] [Info] Start training from score -0.011667
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

In [109]:
train_level1 = pd.concat([t1,tr2,tr3,tr4,tr5],axis=1)
train_level1.columns = ['pred1','pred2','pred3','pred4','pred5']
test_level1 = pd.concat([pr1,pr2,pr3,pr4,pr5],axis=1)
test_level1.columns = ['pred1','pred2','pred3','pred4','pred5']

In [110]:
train_level1

,pred1,pred2,pred3,pred4,pred5
0,0.720029,0.675270,0.669355,0.653113,0.736521
1,0.362419,0.393500,0.330245,0.285226,0.325856
2,0.554296,0.584156,0.588840,0.578370,0.585164
3,0.592314,0.458021,0.465766,0.574414,0.554540
4,0.890592,0.883656,0.850056,0.872023,0.870235
...,...,...,...,...,...
19995,0.786609,0.760280,0.853510,0.799467,0.821233
19996,0.394229,0.404232,0.399834,0.390173,0.449792
19997,0.834150,0.919071,0.920039,0.857982,0.885585
19998,0.428631,0.435263,0.419798,0.426531,0.439186


In [111]:
params = {
    "objective" : "binary",
    "metric" : "binary_logloss",
    "random_state":10,
    "learning_rate":0.05,
    "max_depth":4,
    
    'lambda_l1':2
}
num_round = 150

train_level1_x,val_level1_x,train_level1_y,val_level1_y = tts(train_level1,data_y,test_size=0.25,random_state=10)

lgb_train = lgb.Dataset(train_level1_x,train_level1_y)
lgb_val = lgb.Dataset(val_level1_x,val_level1_y)

# train
model = lgb.train(params,lgb_train,num_boost_round=num_round, valid_names=['train', 'valid']
                  , valid_sets=[lgb_train, lgb_val], callbacks=[lgb.early_stopping(stopping_rounds=20)])

[LightGBM] [Info] Number of positive: 7483, number of negative: 7517
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001117 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1275
[LightGBM] [Info] Number of data points in the train set: 15000, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.498867 -> initscore=-0.004533
[LightGBM] [Info] Start training from score -0.004533
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

In [112]:
blend = model.predict(test_level1)
submission["pred"] = blend
submission.reset_index().to_csv("ensembled.csv",index = False)

array([0.54389587, 0.64242343, 0.26679824, ..., 0.45738002, 0.45868445,
       0.36367027])